<a href="https://colab.research.google.com/github/Mogaz611/Ruppin-Connect4-AI-2025/blob/main/Ruppin_Connect4_AI_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Connect Four AI Tournament - Complete Implementation
Features:
- Multiple AI algorithms (Minimax, Monte Carlo, Genetic, Random)
- Two power-ups per player: Top Gone and Beheading
- Automatic AI play with 0.8s delay
- Human vs Human, Human vs AI, AI vs AI modes
- Configurable board size
- Named AI agents with difficulty levels
"""

import numpy as np
import random
import time
import math
import gradio as gr
from typing import List, Tuple, Optional, Dict

# ======================== BLOCK 1: IMPORTS AND GLOBAL STATE ========================
print("🎮 Connect Four AI Tournament - Starting...")
print("📦 Loading dependencies...")

# Global game state
game = None
ai_agent_1 = None
ai_agent_2 = None
game_mode = "human_vs_human"
game_active = False
auto_play_active = True  # Flag for automatic AI play

# ======================== BLOCK 2: CORE GAME LOGIC ========================
class ConnectFourGUI:
    """Enhanced Connect Four game with fixed win detection and power-ups"""

    def __init__(self, rows=6, cols=7):
        self.rows = rows
        self.cols = cols
        self.board = np.zeros((rows, cols), dtype=int)
        self.current_player = 1
        self.game_over = False
        self.winner = None
        self.move_history = []
        self.power_ups_available = {
            1: {'top_gone': True, 'beheading': True},
            2: {'top_gone': True, 'beheading': True}
        }  # Each player has both power-ups
        self.power_up_history = []  # Track power-up usage

    def reset_game(self):
        """Reset the game to initial state"""
        self.board = np.zeros((self.rows, self.cols), dtype=int)
        self.current_player = 1
        self.game_over = False
        self.winner = None
        self.move_history = []
        self.power_ups_available = {
            1: {'top_gone': True, 'beheading': True},
            2: {'top_gone': True, 'beheading': True}
        }  # Each player has both power-ups
        self.power_up_history = []  # Track power-up usage

    def make_move(self, col: int) -> bool:
        """Make a move in the specified column"""
        if self.game_over or not self.is_valid_move(col):
            return False

        # Find the lowest empty row
        for row in range(self.rows - 1, -1, -1):
            if self.board[row][col] == 0:
                self.board[row][col] = self.current_player
                self.move_history.append((row, col, self.current_player))

                # Check for win
                if self.check_win(row, col):
                    self.game_over = True
                    self.winner = self.current_player
                elif self.is_board_full():
                    self.game_over = True
                    self.winner = 0  # Draw
                else:
                    # Switch players
                    self.current_player = 3 - self.current_player

                return True

        return False

    def is_valid_move(self, col: int) -> bool:
        """Check if a move is valid"""
        return 0 <= col < self.cols and self.board[0][col] == 0

    def get_valid_moves(self) -> list:
        """Get list of valid columns"""
        return [col for col in range(self.cols) if self.is_valid_move(col)]

    def is_board_full(self) -> bool:
        """Check if board is full"""
        return np.all(self.board != 0)

    def check_win(self, last_row: int, last_col: int) -> bool:
        """Check if the last move resulted in a win"""
        player = self.board[last_row][last_col]

        # Horizontal
        count = 1
        # Check left
        col = last_col - 1
        while col >= 0 and self.board[last_row][col] == player:
            count += 1
            col -= 1
        # Check right
        col = last_col + 1
        while col < self.cols and self.board[last_row][col] == player:
            count += 1
            col += 1
        if count >= 4:
            return True

        # Vertical
        count = 1
        # Check down
        row = last_row + 1
        while row < self.rows and self.board[row][last_col] == player:
            count += 1
            row += 1
        # Check up
        row = last_row - 1
        while row >= 0 and self.board[row][last_col] == player:
            count += 1
            row -= 1
        if count >= 4:
            return True

        # Diagonal (top-left to bottom-right)
        count = 1
        # Check down-right
        row, col = last_row + 1, last_col + 1
        while row < self.rows and col < self.cols and self.board[row][col] == player:
            count += 1
            row += 1
            col += 1
        # Check up-left
        row, col = last_row - 1, last_col - 1
        while row >= 0 and col >= 0 and self.board[row][col] == player:
            count += 1
            row -= 1
            col -= 1
        if count >= 4:
            return True

        # Diagonal (top-right to bottom-left)
        count = 1
        # Check down-left
        row, col = last_row + 1, last_col - 1
        while row < self.rows and col >= 0 and self.board[row][col] == player:
            count += 1
            row += 1
            col -= 1
        # Check up-right
        row, col = last_row - 1, last_col + 1
        while row >= 0 and col < self.cols and self.board[row][col] == player:
            count += 1
            row -= 1
            col += 1
        if count >= 4:
            return True

        return False

    def get_board_copy(self):
        """Create a copy of the current game state"""
        new_game = ConnectFourGUI(self.rows, self.cols)
        new_game.board = self.board.copy()
        new_game.current_player = self.current_player
        new_game.game_over = self.game_over
        new_game.winner = self.winner
        new_game.move_history = self.move_history.copy()
        new_game.power_ups_available = {
            1: self.power_ups_available[1].copy(),
            2: self.power_ups_available[2].copy()
        }
        new_game.power_up_history = self.power_up_history.copy()
        return new_game

    def get_top_piece_row(self, col: int) -> int:
        """Get the row of the top piece in a column, or -1 if empty"""
        for row in range(self.rows):
            if self.board[row][col] != 0:
                return row
        return -1

    def use_power_up_option1(self, col1: int, col2: int) -> bool:
        """Use power-up option 1: Remove top piece from 2 columns"""
        if not self.power_ups_available[self.current_player]['top_gone']:
            return False

        if col1 == col2 or col1 < 0 or col1 >= self.cols or col2 < 0 or col2 >= self.cols:
            return False

        row1 = self.get_top_piece_row(col1)
        row2 = self.get_top_piece_row(col2)

        if row1 == -1 or row2 == -1:
            return False

        # Remove the pieces
        removed_pieces = [
            (row1, col1, self.board[row1][col1]),
            (row2, col2, self.board[row2][col2])
        ]

        self.board[row1][col1] = 0
        self.board[row2][col2] = 0

        # Mark power-up as used
        self.power_ups_available[self.current_player]['top_gone'] = False
        self.power_up_history.append({
            'player': self.current_player,
            'type': 'option1',
            'columns': [col1, col2],
            'removed': removed_pieces
        })

        return True

    def use_power_up_option2(self, col: int) -> bool:
        """Use power-up option 2: Remove top 2 pieces from 1 column"""
        if not self.power_ups_available[self.current_player]['top_gone']:
            return False

        if col < 0 or col >= self.cols:
            return False

        row1 = self.get_top_piece_row(col)
        if row1 == -1:
            return False

        # Check if there's a second piece
        row2 = -1
        if row1 + 1 < self.rows and self.board[row1 + 1][col] != 0:
            row2 = row1 + 1

        if row2 == -1:
            return False

        # Remove the pieces
        removed_pieces = [
            (row1, col, self.board[row1][col]),
            (row2, col, self.board[row2][col])
        ]

        self.board[row1][col] = 0
        self.board[row2][col] = 0

        # Mark power-up as used
        self.power_ups_available[self.current_player]['top_gone'] = False
        self.power_up_history.append({
            'player': self.current_player,
            'type': 'option2',
            'columns': [col],
            'removed': removed_pieces
        })

        return True

    def use_beheading_power_up(self) -> bool:
        """Use beheading power-up: Remove top piece from all columns"""
        if not self.power_ups_available[self.current_player]['beheading']:
            return False

        removed_pieces = []
        columns_affected = []

        # Remove top piece from each non-empty column
        for col in range(self.cols):
            row = self.get_top_piece_row(col)
            if row != -1:
                removed_pieces.append((row, col, self.board[row][col]))
                columns_affected.append(col)
                self.board[row][col] = 0

        if not removed_pieces:
            return False  # No pieces to remove

        # Mark power-up as used
        self.power_ups_available[self.current_player]['beheading'] = False
        self.power_up_history.append({
            'player': self.current_player,
            'type': 'beheading',
            'columns': columns_affected,
            'removed': removed_pieces
        })

        return True

    def get_valid_power_up_moves(self) -> dict:
        """Get all valid power-up moves for the current player"""
        result = {'option1': [], 'option2': []}

        if not self.power_ups_available[self.current_player]['top_gone']:
            return result

        # Option 1: Remove top from 2 columns
        non_empty_cols = [col for col in range(self.cols) if self.get_top_piece_row(col) != -1]
        for i in range(len(non_empty_cols)):
            for j in range(i + 1, len(non_empty_cols)):
                result['option1'].append((non_empty_cols[i], non_empty_cols[j]))

        # Option 2: Remove top 2 from 1 column
        for col in range(self.cols):
            row = self.get_top_piece_row(col)
            if row != -1 and row + 1 < self.rows and self.board[row + 1][col] != 0:
                result['option2'].append(col)

        return result

    def get_board_html(self, highlight_last_move=True):
        """Generate HTML representation of the board"""
        player_chars = ['🔴', '🟡']

        html = '<div style="display: flex; flex-direction: column; align-items: center; font-family: Arial;">'

        # Column numbers
        html += '<div style="display: flex; margin-bottom: 10px;">'
        for col in range(self.cols):
            html += f'<div style="width: 60px; text-align: center; font-weight: bold; font-size: 16px;">{col}</div>'
        html += '</div>'

        # Board
        html += '<div style="background-color: #1e3a8a; padding: 10px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.3);">'

        for row in range(self.rows):
            html += '<div style="display: flex;">'
            for col in range(self.cols):
                cell_value = self.board[row][col]

                if cell_value == 0:
                    content = '⚪'
                    bg_color = '#1e3a8a'
                elif cell_value == 1:
                    content = player_chars[0]
                    bg_color = '#1e3a8a'
                else:
                    content = player_chars[1]
                    bg_color = '#1e3a8a'

                html += f'''
                <div style="
                    width: 60px;
                    height: 60px;
                    display: flex;
                    align-items: center;
                    justify-content: center;
                    font-size: 30px;
                    background-color: {bg_color};
                    border: 2px solid #374151;
                    border-radius: 50%;
                    margin: 2px;
                    box-shadow: inset 0 2px 4px rgba(0,0,0,0.1);
                ">{content}</div>
                '''
            html += '</div>'

        html += '</div>'

        # Power-up status
        html += '<div style="display: flex; justify-content: space-around; margin-top: 15px;">'
        for player in [1, 2]:
            # Top Gone status
            tg_status = "✨ Available" if self.power_ups_available[player]['top_gone'] else "❌ Used"
            tg_color = "#16a34a" if self.power_ups_available[player]['top_gone'] else "#6b7280"

            # Beheading status
            bh_status = "⚔️ Available" if self.power_ups_available[player]['beheading'] else "❌ Used"
            bh_color = "#9333ea" if self.power_ups_available[player]['beheading'] else "#6b7280"

            html += f'''
            <div style="display: flex; flex-direction: column; gap: 5px;">
                <div style="
                    padding: 10px;
                    background-color: {tg_color}15;
                    border: 2px solid {tg_color};
                    border-radius: 8px;
                    color: {tg_color};
                    font-weight: bold;
                ">
                    {player_chars[player-1]} P{player} Top Gone: {tg_status}
                </div>
                <div style="
                    padding: 10px;
                    background-color: {bh_color}15;
                    border: 2px solid {bh_color};
                    border-radius: 8px;
                    color: {bh_color};
                    font-weight: bold;
                ">
                    {player_chars[player-1]} P{player} Beheading: {bh_status}
                </div>
            </div>
            '''
        html += '</div>'

        # Game status
        if self.game_over:
            if self.winner == 0:
                status = "🤝 <strong>It's a Draw!</strong>"
                status_color = "#6b7280"
            else:
                winner_char = player_chars[self.winner - 1]
                status = f"🎉 <strong>Player {self.winner} {winner_char} Wins!</strong>"
                status_color = "#16a34a"
        else:
            current_char = player_chars[self.current_player - 1]
            status = f"🎯 <strong>Player {self.current_player} {current_char} Turn</strong>"
            status_color = "#2563eb"

        html += f'''
        <div style="
            margin-top: 20px;
            padding: 15px;
            background-color: {status_color}15;
            border: 2px solid {status_color};
            border-radius: 8px;
            color: {status_color};
            font-size: 18px;
            text-align: center;
        ">{status}</div>
        '''

        html += '</div>'
        return html

print("✅ Core game logic with FIXED win detection and Top Gone + Beheading power-ups implemented successfully!")

# ======================== BLOCK 3: AI ALGORITHMS ========================

# Base AI Agent
class BaseAgent:
    """Base class for AI agents"""
    def __init__(self, player_id: int, name: str):
        self.player_id = player_id
        self.opponent_id = 3 - player_id
        self.name = name

    def get_move(self, game) -> int:
        """Get the next move (to be implemented by subclasses)"""
        raise NotImplementedError

# Minimax Agent with Alpha-Beta Pruning
class MinimaxAgent(BaseAgent):
    """AI agent using Minimax algorithm with alpha-beta pruning"""

    def __init__(self, player_id: int, depth: int = 4, difficulty: str = "Medium"):
        # Create descriptive name based on difficulty
        names = {
            "Easy": "Minimax Novice",
            "Medium": "Minimax Strategist",
            "Hard": "Minimax Expert",
            "Expert": "Minimax Grandmaster"
        }
        super().__init__(player_id, names.get(difficulty, "Minimax AI"))

        # Set depth based on difficulty
        depth_map = {"Easy": 2, "Medium": 4, "Hard": 5, "Expert": 6}
        self.depth = depth_map.get(difficulty, 4)
        self.nodes_explored = 0

    def get_move(self, game) -> int:
        """Get the best move using minimax"""
        self.nodes_explored = 0

        # First check if we should use power-up
        if game.power_ups_available[self.player_id]['top_gone'] or game.power_ups_available[self.player_id]['beheading']:
            power_up_move = self.evaluate_power_up_use(game)
            if power_up_move:
                return power_up_move

        _, best_col = self.minimax(game, self.depth, float('-inf'), float('inf'), True)
        return best_col if best_col != -1 else random.choice(game.get_valid_moves())

    def evaluate_power_up_use(self, game):
        """Evaluate if power-up should be used (returns None for regular move, or power-up move dict)"""
        # Check if opponent is about to win
        opponent_winning_cols = []
        for col in game.get_valid_moves():
            game_copy = game.get_board_copy()
            game_copy.current_player = self.opponent_id
            game_copy.make_move(col)
            if game_copy.winner == self.opponent_id:
                opponent_winning_cols.append(col)

        if opponent_winning_cols:
            # First try beheading if it would help
            if game.power_ups_available[self.player_id]['beheading']:
                non_empty_cols = sum(1 for col in range(game.cols) if game.get_top_piece_row(col) != -1)
                if non_empty_cols >= 4:  # Use beheading if many columns have pieces
                    return {'type': 'power_up', 'option': 'beheading'}

            # Try to disrupt opponent's winning position with top gone power-up
            if game.power_ups_available[self.player_id]['top_gone']:
                valid_power_ups = game.get_valid_power_up_moves()

                # Check option 2 first (remove 2 from 1 column)
                for col in valid_power_ups['option2']:
                    if col in opponent_winning_cols:
                        return {'type': 'power_up', 'option': 2, 'columns': [col]}

                # Then check option 1
                for col1, col2 in valid_power_ups['option1']:
                    if col1 in opponent_winning_cols or col2 in opponent_winning_cols:
                        return {'type': 'power_up', 'option': 1, 'columns': [col1, col2]}

        return None

    def minimax(self, game, depth: int, alpha: float, beta: float, maximizing: bool):
        """Minimax algorithm with alpha-beta pruning"""
        self.nodes_explored += 1

        if depth == 0 or game.game_over:
            return self.evaluate_board(game), -1

        valid_moves = game.get_valid_moves()
        if not valid_moves:
            return 0, -1

        best_col = random.choice(valid_moves)

        if maximizing:
            max_eval = float('-inf')
            for col in valid_moves:
                game_copy = game.get_board_copy()
                game_copy.make_move(col)
                eval_score, _ = self.minimax(game_copy, depth - 1, alpha, beta, False)

                if eval_score > max_eval:
                    max_eval = eval_score
                    best_col = col

                alpha = max(alpha, eval_score)
                if beta <= alpha:
                    break

            return max_eval, best_col
        else:
            min_eval = float('inf')
            for col in valid_moves:
                game_copy = game.get_board_copy()
                game_copy.make_move(col)
                eval_score, _ = self.minimax(game_copy, depth - 1, alpha, beta, True)

                if eval_score < min_eval:
                    min_eval = eval_score
                    best_col = col

                beta = min(beta, eval_score)
                if beta <= alpha:
                    break

            return min_eval, best_col

    def evaluate_board(self, game):
        """Enhanced board evaluation function"""
        if game.game_over:
            if game.winner == self.player_id:
                return 10000
            elif game.winner == self.opponent_id:
                return -10000
            else:
                return 0

        score = 0

        # Evaluate all possible 4-in-a-row positions
        # Horizontal
        for row in range(game.rows):
            for col in range(game.cols - 3):
                window = [game.board[row][col + i] for i in range(4)]
                score += self.evaluate_window(window)

        # Vertical
        for row in range(game.rows - 3):
            for col in range(game.cols):
                window = [game.board[row + i][col] for i in range(4)]
                score += self.evaluate_window(window)

        # Positive diagonal
        for row in range(game.rows - 3):
            for col in range(game.cols - 3):
                window = [game.board[row + i][col + i] for i in range(4)]
                score += self.evaluate_window(window)

        # Negative diagonal
        for row in range(3, game.rows):
            for col in range(game.cols - 3):
                window = [game.board[row - i][col + i] for i in range(4)]
                score += self.evaluate_window(window)

        # Center column preference
        center_col = game.cols // 2
        for row in range(game.rows):
            if game.board[row][center_col] == self.player_id:
                score += 3

        return score

    def evaluate_window(self, window):
        """Evaluate a 4-piece window"""
        my_count = window.count(self.player_id)
        opp_count = window.count(self.opponent_id)
        empty_count = window.count(0)

        if my_count == 4:
            return 100
        elif my_count == 3 and empty_count == 1:
            return 5
        elif my_count == 2 and empty_count == 2:
            return 2
        elif opp_count == 3 and empty_count == 1:
            return -4
        elif opp_count == 2 and empty_count == 2:
            return -1

        return 0

# Monte Carlo Tree Search Agent
class MonteCarloAgent(BaseAgent):
    """AI agent using Monte Carlo simulations"""

    def __init__(self, player_id: int, simulations: int = 100, difficulty: str = "Medium"):
        # Create descriptive name based on difficulty
        names = {
            "Easy": "Monte Carlo Explorer",
            "Medium": "Monte Carlo Analyst",
            "Hard": "Monte Carlo Strategist",
            "Expert": "Monte Carlo Master"
        }
        super().__init__(player_id, names.get(difficulty, "Monte Carlo AI"))

        # Set simulations based on difficulty
        sim_map = {"Easy": 50, "Medium": 100, "Hard": 200, "Expert": 500}
        self.simulations = sim_map.get(difficulty, 100)

    def get_move(self, game) -> int:
        """Get best move using Monte Carlo simulations"""
        # First check if we should use power-up
        if (game.power_ups_available[self.player_id]['top_gone'] or
            game.power_ups_available[self.player_id]['beheading']) and random.random() < 0.3:  # 30% chance to consider power-up
            power_up_move = self.evaluate_power_up_use(game)
            if power_up_move:
                return power_up_move

        valid_moves = game.get_valid_moves()
        if not valid_moves:
            return -1

        move_scores = {}

        for move in valid_moves:
            total_score = 0

            for _ in range(self.simulations // len(valid_moves)):
                game_copy = game.get_board_copy()
                game_copy.make_move(move)
                score = self.simulate_random_game(game_copy)
                total_score += score

            move_scores[move] = total_score

        best_move = max(move_scores.keys(), key=lambda x: move_scores[x])
        return best_move

    def evaluate_power_up_use(self, game):
        """Simple power-up evaluation for Monte Carlo agent"""
        valid_power_ups = game.get_valid_power_up_moves()

        # Randomly choose to use power-up if options available
        if game.power_ups_available[self.player_id]['beheading'] and random.random() < 0.2:
            return {'type': 'power_up', 'option': 'beheading'}
        elif valid_power_ups['option2'] and random.random() < 0.5:
            col = random.choice(valid_power_ups['option2'])
            return {'type': 'power_up', 'option': 2, 'columns': [col]}
        elif valid_power_ups['option1']:
            cols = random.choice(valid_power_ups['option1'])
            return {'type': 'power_up', 'option': 1, 'columns': list(cols)}

        return None

    def simulate_random_game(self, game):
        """Simulate a random game and return the outcome"""
        sim_game = game.get_board_copy()

        while not sim_game.game_over:
            valid_moves = sim_game.get_valid_moves()
            if not valid_moves:
                break

            move = random.choice(valid_moves)
            sim_game.make_move(move)

        if sim_game.winner == self.player_id:
            return 1
        elif sim_game.winner == self.opponent_id:
            return -1
        else:
            return 0

# Genetic Algorithm Agent
class GeneticAgent(BaseAgent):
    """AI agent using genetic algorithm"""

    def __init__(self, player_id: int, population_size: int = 20, generations: int = 20,
                 mutation_rate: float = 0.1, difficulty: str = "Medium"):
        # Create descriptive name based on difficulty/parameters
        if generations <= 15:
            name = "Genetic Pioneer"
        elif generations <= 25:
            name = "Genetic Evolutionist"
        else:
            name = "Genetic Apex"

        super().__init__(player_id, name)
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate

    def get_move(self, game) -> int:
        """Get best move using genetic algorithm"""
        # First check if we should use power-up (genetic agents rarely use it)
        if (game.power_ups_available[self.player_id]['top_gone'] or
            game.power_ups_available[self.player_id]['beheading']) and random.random() < 0.1:  # 10% chance
            valid_power_ups = game.get_valid_power_up_moves()
            if game.power_ups_available[self.player_id]['beheading'] and random.random() < 0.3:
                return {'type': 'power_up', 'option': 'beheading'}
            elif valid_power_ups['option2']:
                col = random.choice(valid_power_ups['option2'])
                return {'type': 'power_up', 'option': 2, 'columns': [col]}

        valid_moves = game.get_valid_moves()
        if not valid_moves:
            return -1

        # Initialize population
        population = [self.create_individual(game) for _ in range(self.population_size)]

        # Evolve population
        for generation in range(self.generations):
            # Evaluate fitness for each move
            move_fitness = {}

            for move in valid_moves:
                game_copy = game.get_board_copy()
                game_copy.make_move(move)

                fitness_scores = []
                for individual in population:
                    fitness = self.evaluate_individual(individual, game_copy)
                    fitness_scores.append(fitness)

                move_fitness[move] = sum(fitness_scores) / len(fitness_scores)

            # Create new population
            new_population = []
            fitness_values = [self.evaluate_individual(ind, game) for ind in population]

            # Selection and reproduction
            for _ in range(self.population_size):
                parent1 = self.tournament_selection(population, fitness_values)
                parent2 = self.tournament_selection(population, fitness_values)
                child = self.crossover(parent1, parent2)
                child = self.mutate(child)
                new_population.append(child)

            population = new_population

        # Return best move based on final population
        move_scores = {}
        for move in valid_moves:
            game_copy = game.get_board_copy()
            game_copy.make_move(move)

            scores = [self.evaluate_individual(ind, game_copy) for ind in population]
            move_scores[move] = sum(scores) / len(scores)

        best_move = max(move_scores.keys(), key=lambda x: move_scores[x])
        return best_move

    def create_individual(self, game):
        """Create a random individual (weight vector)"""
        # Weights for: center control, mobility, patterns, blocking
        return [random.uniform(-1, 1) for _ in range(4)]

    def evaluate_individual(self, individual, game):
        """Evaluate fitness of an individual on a game state"""
        if game.game_over:
            if game.winner == self.player_id:
                return 100
            elif game.winner == self.opponent_id:
                return -100
            else:
                return 0

        features = self.extract_features(game)
        fitness = sum(w * f for w, f in zip(individual, features))
        return fitness

    def extract_features(self, game):
        """Extract features from game state"""
        features = []

        # Center control
        center_col = game.cols // 2
        center_control = 0
        for row in range(game.rows):
            if game.board[row][center_col] == self.player_id:
                center_control += 1
            elif game.board[row][center_col] == self.opponent_id:
                center_control -= 1
        features.append(center_control)

        # Mobility (number of valid moves)
        features.append(len(game.get_valid_moves()))

        # Pattern detection (simplified)
        pattern_score = 0
        for row in range(game.rows):
            for col in range(game.cols - 2):
                if (game.board[row][col] == self.player_id and
                    game.board[row][col + 1] == self.player_id):
                    pattern_score += 1
        features.append(pattern_score)

        # Blocking opponent patterns
        block_score = 0
        for row in range(game.rows):
            for col in range(game.cols - 2):
                if (game.board[row][col] == self.opponent_id and
                    game.board[row][col + 1] == self.opponent_id):
                    block_score += 1
        features.append(block_score)

        return features

    def tournament_selection(self, population, fitness_values):
        """Select individual using tournament selection"""
        tournament_size = 3
        tournament_indices = random.sample(range(len(population)), tournament_size)
        tournament_fitness = [fitness_values[i] for i in tournament_indices]
        winner_idx = tournament_indices[tournament_fitness.index(max(tournament_fitness))]
        return population[winner_idx]

    def crossover(self, parent1, parent2):
        """Create offspring through crossover"""
        crossover_point = random.randint(1, len(parent1) - 1)
        child = parent1[:crossover_point] + parent2[crossover_point:]
        return child

    def mutate(self, individual):
        """Mutate individual"""
        mutated = individual.copy()
        for i in range(len(mutated)):
            if random.random() < self.mutation_rate:
                mutated[i] += random.uniform(-0.5, 0.5)
                mutated[i] = max(-1, min(1, mutated[i]))  # Clamp to [-1, 1]
        return mutated

# Random Agent (Baseline)
class RandomAgent(BaseAgent):
    """Random AI agent for baseline comparison"""

    def __init__(self, player_id: int, difficulty: str = "Easy"):
        super().__init__(player_id, "Random Baseline")

    def get_move(self, game) -> int:
        """Get random valid move"""
        # Randomly decide to use power-up (5% chance)
        if (game.power_ups_available[self.player_id]['top_gone'] or
            game.power_ups_available[self.player_id]['beheading']) and random.random() < 0.05:

            if game.power_ups_available[self.player_id]['beheading'] and random.random() < 0.3:
                return {'type': 'power_up', 'option': 'beheading'}

            valid_power_ups = game.get_valid_power_up_moves()
            if random.random() < 0.5 and valid_power_ups['option1']:
                cols = random.choice(valid_power_ups['option1'])
                return {'type': 'power_up', 'option': 1, 'columns': list(cols)}
            elif valid_power_ups['option2']:
                col = random.choice(valid_power_ups['option2'])
                return {'type': 'power_up', 'option': 2, 'columns': [col]}

        valid_moves = game.get_valid_moves()
        return random.choice(valid_moves) if valid_moves else -1

print("✅ All AI algorithms with descriptive names and power-up support implemented successfully!")

# ======================== BLOCK 4: GAME MANAGEMENT ========================

def create_ai_agent(algorithm: str, difficulty: str, player_id: int,
                   generations: int = 20, population: int = 20):
    """Factory function to create AI agents"""
    if algorithm == "Minimax":
        return MinimaxAgent(player_id, difficulty=difficulty)
    elif algorithm == "Monte Carlo":
        return MonteCarloAgent(player_id, difficulty=difficulty)
    elif algorithm == "Genetic":
        return GeneticAgent(player_id, generations=generations,
                          population_size=population, difficulty=difficulty)
    elif algorithm == "Random":
        return RandomAgent(player_id)
    else:
        return None

def setup_game(rows, cols, mode, algo1, diff1, gen1, pop1, algo2, diff2, gen2, pop2):
    """Setup game with all parameters"""
    global game, ai_agent_1, ai_agent_2, game_mode, game_active, auto_play_active

    game = ConnectFourGUI(int(rows), int(cols))
    game_mode = mode
    game_active = True
    auto_play_active = True

    # Create AI agents based on mode
    if mode == "human_vs_human":
        ai_agent_1 = None
        ai_agent_2 = None
    elif mode == "human_vs_ai":
        ai_agent_1 = None
        if algo2 != "Human":
            ai_agent_2 = create_ai_agent(algo2, diff2, 2, gen2, pop2)
        else:
            ai_agent_2 = None
    elif mode == "ai_vs_human":
        if algo1 != "Human":
            ai_agent_1 = create_ai_agent(algo1, diff1, 1, gen1, pop1)
        else:
            ai_agent_1 = None
        ai_agent_2 = None
    elif mode == "ai_vs_ai":
        if algo1 != "Human":
            ai_agent_1 = create_ai_agent(algo1, diff1, 1, gen1, pop1)
        else:
            ai_agent_1 = None
        if algo2 != "Human":
            ai_agent_2 = create_ai_agent(algo2, diff2, 2, gen2, pop2)
        else:
            ai_agent_2 = None

    status = f"🎮 New {rows}×{cols} game started!\n"
    status += f"Mode: {mode.replace('_', ' ').title()}\n"

    if ai_agent_1:
        status += f"Player 1: {ai_agent_1.name}\n"
    else:
        status += "Player 1: Human Player\n"

    if ai_agent_2:
        status += f"Player 2: {ai_agent_2.name}\n"
    else:
        status += "Player 2: Human Player\n"

    # Check if AI should make first move
    current_agent = ai_agent_1 if game.current_player == 1 else ai_agent_2
    should_ai_play = current_agent is not None

    if should_ai_play:
        status += f"\n🎯 {current_agent.name} will play automatically in 0.8s..."

    return game.get_board_html(), status, should_ai_play

def make_ai_move():
    """Make one AI move"""
    global game, ai_agent_1, ai_agent_2, game_active

    if not game_active or game.game_over:
        return game.get_board_html(), "Game is over! Start a new game.", False

    current_agent = ai_agent_1 if game.current_player == 1 else ai_agent_2

    if current_agent is None:
        return game.get_board_html(), "It's human player's turn!", False

    # Make AI move
    start_time = time.time()
    ai_move = current_agent.get_move(game)
    end_time = time.time()

    # Check if it's a power-up move
    if isinstance(ai_move, dict) and ai_move.get('type') == 'power_up':
        if ai_move['option'] == 'beheading':
            success = game.use_beheading_power_up()
            status = f"⚔️ {current_agent.name} used Beheading power-up! Removed top piece from all columns"
        elif ai_move['option'] == 1:
            success = game.use_power_up_option1(ai_move['columns'][0], ai_move['columns'][1])
            status = f"✨ {current_agent.name} used Top Gone power-up! Removed top pieces from columns {ai_move['columns'][0]} and {ai_move['columns'][1]}"
        else:
            success = game.use_power_up_option2(ai_move['columns'][0])
            status = f"✨ {current_agent.name} used Top Gone power-up! Removed top 2 pieces from column {ai_move['columns'][0]}"
        status += f" ({end_time - start_time:.2f}s)"
    else:
        # Regular move
        if ai_move != -1:
            game.make_move(ai_move)

        status = f"🤖 {current_agent.name} played column {ai_move} ({end_time - start_time:.2f}s)"

        if hasattr(current_agent, 'nodes_explored'):
            status += f" | {current_agent.nodes_explored} nodes"

    if game.game_over:
        game_active = False
        if game.winner == 0:
            status += " | 🤝 Draw!"
        else:
            status += f" | 🎉 Player {game.winner} wins!"
        return game.get_board_html(), status, False
    else:
        # Show whose turn is next
        next_agent = ai_agent_1 if game.current_player == 1 else ai_agent_2
        if next_agent:
            status += f"\n🎯 Next: {next_agent.name} - Auto-play in 0.8s..."
            # Return True to indicate another AI should play
            return game.get_board_html(), status, True
        else:
            status += f"\n👤 Next: Human Player - Click a column"
            return game.get_board_html(), status, False

def make_human_move(col):
    """Handle human move"""
    global game, ai_agent_1, ai_agent_2, game_active

    if not game_active or game.game_over:
        return game.get_board_html(), "Game is over! Start a new game.", False

    # Check if it's human's turn
    current_agent = ai_agent_1 if game.current_player == 1 else ai_agent_2
    if current_agent is not None:
        return game.get_board_html(), f"🤖 It's {current_agent.name}'s turn! Please wait...", False

    if col >= game.cols or not game.is_valid_move(col):
        return game.get_board_html(), f"❌ Invalid move! Column {col}", False

    # Make human move
    game.make_move(col)
    status = f"👤 You played column {col}!"

    if game.game_over:
        game_active = False
        if game.winner == 0:
            status += " 🤝 Draw!"
        else:
            status += f" 🎉 Player {game.winner} wins!"
        return game.get_board_html(), status, False
    else:
        # Show whose turn is next
        next_agent = ai_agent_1 if game.current_player == 1 else ai_agent_2
        if next_agent:
            status += f"\n🎯 Next: {next_agent.name} - Auto-play in 0.8s..."
            # Return True to indicate AI should play next
            return game.get_board_html(), status, True
        else:
            status += f"\n👤 Next: Human Player"
            return game.get_board_html(), status, False

def use_human_power_up(option, col1, col2=None):
    """Handle human power-up usage"""
    global game, ai_agent_1, ai_agent_2, game_active

    if not game_active or game.game_over:
        return game.get_board_html(), "Game is over! Start a new game.", False

    # Check if it's human's turn
    current_agent = ai_agent_1 if game.current_player == 1 else ai_agent_2
    if current_agent is not None:
        return game.get_board_html(), f"🤖 It's {current_agent.name}'s turn!", False

    success = False
    if option == 'beheading':
        if not game.power_ups_available[game.current_player]['beheading']:
            return game.get_board_html(), "❌ You've already used your Beheading power-up!", False
        success = game.use_beheading_power_up()
        if success:
            status = f"⚔️ You used Beheading power-up! Removed top piece from all columns"
        else:
            status = "❌ No pieces to remove!"
    elif option == 1 and col2 is not None:
        if not game.power_ups_available[game.current_player]['top_gone']:
            return game.get_board_html(), "❌ You've already used your Top Gone power-up!", False
        success = game.use_power_up_option1(col1, col2)
        if success:
            status = f"✨ You used Top Gone power-up! Removed top pieces from columns {col1} and {col2}"
    elif option == 2:
        if not game.power_ups_available[game.current_player]['top_gone']:
            return game.get_board_html(), "❌ You've already used your Top Gone power-up!", False
        success = game.use_power_up_option2(col1)
        if success:
            status = f"✨ You used Top Gone power-up! Removed top 2 pieces from column {col1}"

    if not success:
        return game.get_board_html(), "❌ Invalid power-up usage! Check the columns.", False

    # Check for game over (unlikely but possible)
    if game.game_over:
        game_active = False
        if game.winner == 0:
            status += " | 🤝 Draw!"
        else:
            status += f" | 🎉 Player {game.winner} wins!"
        return game.get_board_html(), status, False
    else:
        # Show whose turn is next (still current player after power-up)
        status += f"\n🎯 Still your turn - make a regular move!"
        return game.get_board_html(), status, False

def check_for_ai_turn():
    """Check if it's AI's turn and return whether AI should play"""
    global game, ai_agent_1, ai_agent_2, game_active, auto_play_active

    if not game_active or game.game_over or not auto_play_active:
        return False

    current_agent = ai_agent_1 if game.current_player == 1 else ai_agent_2
    return current_agent is not None

def reset_game():
    """Reset the current game"""
    global game, game_active
    game.reset_game()
    game_active = True

    return game.get_board_html(), "🔄 Game reset! Configure and start a new game."

def get_analysis():
    """Enhanced game analysis"""
    if not game.move_history:
        return "No moves yet! Start playing to see detailed analysis."

    analysis = f"# 📊 **Game Analysis**\n\n"

    # Game info
    analysis += f"**🎮 Configuration**\n"
    analysis += f"• Board: {game.rows}×{game.cols}\n"
    analysis += f"• Mode: {game_mode}\n"
    analysis += f"• Moves: {len(game.move_history)}\n"

    # Players
    analysis += f"\n**👥 Players**\n"
    if ai_agent_1:
        analysis += f"• Player 1: {ai_agent_1.name}\n"
    else:
        analysis += f"• Player 1: Human Player\n"

    if ai_agent_2:
        analysis += f"• Player 2: {ai_agent_2.name}\n"
    else:
        analysis += f"• Player 2: Human Player\n"

    # Power-up status
    analysis += f"\n**✨ Power-Up Status**\n"
    for player in [1, 2]:
        tg_status = "Available" if game.power_ups_available[player]['top_gone'] else "Used"
        bh_status = "Available" if game.power_ups_available[player]['beheading'] else "Used"
        analysis += f"• Player {player} Top Gone: {tg_status} | Beheading: {bh_status}\n"

    # Power-up history
    if game.power_up_history:
        analysis += f"\n**💥 Power-Up History**\n"
        for i, power_up in enumerate(game.power_up_history):
            player_name = "Human"
            if power_up['player'] == 1 and ai_agent_1:
                player_name = ai_agent_1.name
            elif power_up['player'] == 2 and ai_agent_2:
                player_name = ai_agent_2.name

            if power_up['type'] == 'beheading':
                analysis += f"{i+1}. P{power_up['player']} ({player_name}) used Beheading - removed from {len(power_up['columns'])} columns\n"
            elif power_up['type'] == 'option1':
                analysis += f"{i+1}. P{power_up['player']} ({player_name}) removed top from columns {power_up['columns'][0]}, {power_up['columns'][1]}\n"
            else:
                analysis += f"{i+1}. P{power_up['player']} ({player_name}) removed top 2 from column {power_up['columns'][0]}\n"

    # Game status
    analysis += f"\n**🏆 Status**\n"
    if game.game_over:
        if game.winner == 0:
            analysis += f"• Result: 🤝 Draw\n"
        else:
            analysis += f"• Winner: 🎉 Player {game.winner}\n"
    else:
        current_name = "Human"
        if game.current_player == 1 and ai_agent_1:
            current_name = ai_agent_1.name
        elif game.current_player == 2 and ai_agent_2:
            current_name = ai_agent_2.name
        analysis += f"• Current: Player {game.current_player} ({current_name})\n"

    # Move history
    analysis += f"\n**📝 Move History**\n"
    for i, (row, col, player) in enumerate(game.move_history):
        player_name = "Human"
        if player == 1 and ai_agent_1:
            player_name = ai_agent_1.name
        elif player == 2 and ai_agent_2:
            player_name = ai_agent_2.name

        analysis += f"{i+1:2d}. P{player} ({player_name}) → Col {col}\n"

    # Strategic analysis
    analysis += f"\n**🧠 Strategic Analysis**\n"
    col_usage = [0] * game.cols
    for _, col, _ in game.move_history:
        col_usage[col] += 1

    analysis += f"• Column Usage: "
    for i, count in enumerate(col_usage):
        if count > 0:
            analysis += f"C{i}({count}) "
    analysis += "\n"

    # Center control
    center_col = game.cols // 2
    center_p1 = np.count_nonzero(game.board[:, center_col] == 1)
    center_p2 = np.count_nonzero(game.board[:, center_col] == 2)
    analysis += f"• Center Control: P1({center_p1}) vs P2({center_p2})\n"

    return analysis

print("✅ Game management with automatic AI and power-up system implemented successfully!")

# ======================== BLOCK 5: USER INTERFACE ========================

def create_interface():
    """Create the comprehensive Gradio interface"""

    with gr.Blocks(title="🎮 Connect Four AI - Fixed Edition", theme=gr.themes.Soft()) as demo:

        gr.Markdown("""
        # 🎮 Connect Four AI Tournament
        ## Advanced Algorithms Project with Power-Ups

        **🚀 Features:** Named AI Agents • Top Gone Power-Up • Beheading Power-Up • Automatic AI Play (0.8s delay) • Genetic Evolution
        **✨ Power-Ups:** Each player gets TWO power-ups per game - use them strategically!
        **🤖 Algorithms:** Minimax Alpha-Beta • Monte Carlo Tree Search • Genetic Algorithm • Random Baseline
        """)

        with gr.Row():
            with gr.Column(scale=2):
                # Board setup moved to top
                with gr.Group():
                    gr.Markdown("### 🏗️ Board Setup")
                    with gr.Row():
                        board_rows = gr.Slider(5, 10, value=6, step=1, label="Rows (Height)")
                        board_cols = gr.Slider(5, 10, value=7, step=1, label="Columns (Width)")

                # Game board
                board_display = gr.HTML(value=game.get_board_html(), label="🎯 Game Board")

                # Column buttons
                with gr.Row():
                    col_buttons = []
                    for i in range(10):
                        btn = gr.Button(f"Col {i}", size="sm", visible=(i < 7))
                        col_buttons.append(btn)

                # Control buttons
                with gr.Row():
                    auto_play_toggle = gr.Checkbox(label="🤖 Auto-play AI moves", value=True)
                    next_ai_btn = gr.Button("⏭️ Force AI Move Now", variant="secondary", size="lg")
                    reset_btn = gr.Button("🔄 Reset Game", variant="secondary", size="lg")

                # Power-up controls
                with gr.Group():
                    gr.Markdown("### ✨ Power-Ups (Use Once Per Game)")

                    # Top Gone Power-up
                    with gr.Accordion("🎯 Top Gone Power-Up", open=False):
                        with gr.Row():
                            with gr.Column():
                                gr.Markdown("**Option 1**: Remove top piece from 2 columns")
                                with gr.Row():
                                    power_col1_opt1 = gr.Number(label="Column 1", value=0, precision=0, minimum=0, maximum=9)
                                    power_col2_opt1 = gr.Number(label="Column 2", value=1, precision=0, minimum=0, maximum=9)
                                power_btn_opt1 = gr.Button("✨ Use Option 1", variant="secondary")

                            with gr.Column():
                                gr.Markdown("**Option 2**: Remove top 2 pieces from 1 column")
                                power_col_opt2 = gr.Number(label="Column", value=0, precision=0, minimum=0, maximum=9)
                                power_btn_opt2 = gr.Button("✨ Use Option 2", variant="secondary")

                    # Beheading Power-up
                    with gr.Accordion("⚔️ Beheading Power-Up", open=False):
                        gr.Markdown("**Remove the top piece from ALL columns that have pieces**")
                        gr.Markdown("*This is a powerful move that affects the entire board!*")
                        beheading_btn = gr.Button("⚔️ Use Beheading", variant="primary", size="lg")

                # Status
                status_msg = gr.Textbox(
                    label="📢 Game Status & AI Information",
                    value="🎮 Welcome! AI players will move automatically after 0.8 seconds!\n✨ Each player gets TWO power-ups per game!",
                    lines=5,
                    interactive=False
                )

            with gr.Column(scale=1):
                gr.Markdown("### ⚙️ Player Configuration")

                # Player 1 Configuration
                with gr.Group():
                    gr.Markdown("**🔴 Player 1 - Choose Player Type**")
                    player1_type = gr.Radio(
                        choices=["Human", "Minimax Novice", "Minimax Strategist", "Minimax Expert", "Minimax Grandmaster",
                                "Monte Carlo Explorer", "Monte Carlo Analyst", "Monte Carlo Strategist", "Monte Carlo Master",
                                "Genetic Pioneer", "Genetic Evolutionist", "Genetic Apex", "Random Baseline"],
                        value="Human",
                        label="Player 1 Type"
                    )

                    # Player 1 AI Settings (conditionally visible)
                    with gr.Group(visible=False) as player1_settings:
                        diff1 = gr.Dropdown(
                            choices=["Easy", "Medium", "Hard", "Expert"],
                            value="Medium",
                            label="Difficulty",
                            visible=False
                        )
                        gen1 = gr.Slider(10, 50, value=20, step=5, label="Generations", visible=False)
                        pop1 = gr.Slider(10, 50, value=20, step=5, label="Population Size", visible=False)

                # Player 2 Configuration
                with gr.Group():
                    gr.Markdown("**🟡 Player 2 - Choose Player Type**")
                    player2_type = gr.Radio(
                        choices=["Human", "Minimax Novice", "Minimax Strategist", "Minimax Expert", "Minimax Grandmaster",
                                "Monte Carlo Explorer", "Monte Carlo Analyst", "Monte Carlo Strategist", "Monte Carlo Master",
                                "Genetic Pioneer", "Genetic Evolutionist", "Genetic Apex", "Random Baseline"],
                        value="Monte Carlo Analyst",
                        label="Player 2 Type"
                    )

                    # Player 2 AI Settings (conditionally visible)
                    with gr.Group(visible=True) as player2_settings:
                        diff2 = gr.Dropdown(
                            choices=["Easy", "Medium", "Hard", "Expert"],
                            value="Medium",
                            label="Difficulty",
                            visible=True
                        )
                        gen2 = gr.Slider(10, 50, value=20, step=5, label="Generations", visible=False)
                        pop2 = gr.Slider(10, 50, value=20, step=5, label="Population Size", visible=False)

                # Game controls
                with gr.Group():
                    start_btn = gr.Button("🚀 Start New Game", variant="primary", size="lg")

                    gr.Markdown("""
                    **💡 How to Play:**
                    - AI players move automatically after 0.8s
                    - Toggle auto-play on/off anytime
                    - Human players click column buttons
                    - Use your power-ups strategically!

                    **✨ Power-Ups (One Each Per Game):**
                    - **Top Gone**: Remove pieces from specific columns
                    - **Beheading**: Remove top piece from ALL columns at once!
                    - Power-ups don't end your turn!
                    """)

                # Analysis
                with gr.Accordion("📊 Game Analysis", open=False):
                    analysis_display = gr.Textbox(
                        label="Detailed Analysis",
                        value="🎯 Start a game to see detailed analysis!",
                        lines=12,
                        interactive=False
                    )
                    analysis_btn = gr.Button("🔍 Refresh Analysis", size="sm")

        # Hidden components for algorithm and game mode
        algo1 = gr.Textbox(value="Human", visible=False)
        algo2 = gr.Textbox(value="Monte Carlo", visible=False)
        game_mode_selector = gr.Textbox(value="human_vs_ai", visible=False)
        should_trigger_ai = gr.Checkbox(value=False, visible=False)
        timer = gr.Timer(value=0.8, active=False)

        # Function to update auto-play setting
        def toggle_auto_play(value):
            global auto_play_active
            auto_play_active = value
            return f"Auto-play {'enabled' if value else 'disabled'}"

        # Function to update player settings visibility and algorithm
        def update_player1_settings(player_type):
            if player_type == "Human":
                return {
                    player1_settings: gr.update(visible=False),
                    diff1: gr.update(visible=False),
                    gen1: gr.update(visible=False),
                    pop1: gr.update(visible=False),
                    algo1: "Human",
                    game_mode_selector: determine_game_mode(player_type, player2_type.value)
                }
            elif "Minimax" in player_type:
                difficulty = "Easy" if "Novice" in player_type else "Medium" if "Strategist" in player_type else "Hard" if "Expert" in player_type else "Expert"
                return {
                    player1_settings: gr.update(visible=True),
                    diff1: gr.update(visible=True, value=difficulty),
                    gen1: gr.update(visible=False),
                    pop1: gr.update(visible=False),
                    algo1: "Minimax",
                    game_mode_selector: determine_game_mode(player_type, player2_type.value)
                }
            elif "Monte Carlo" in player_type:
                difficulty = "Easy" if "Explorer" in player_type else "Medium" if "Analyst" in player_type else "Hard" if "Strategist" in player_type else "Expert"
                return {
                    player1_settings: gr.update(visible=True),
                    diff1: gr.update(visible=True, value=difficulty),
                    gen1: gr.update(visible=False),
                    pop1: gr.update(visible=False),
                    algo1: "Monte Carlo",
                    game_mode_selector: determine_game_mode(player_type, player2_type.value)
                }
            elif "Genetic" in player_type:
                gens = 15 if "Pioneer" in player_type else 25 if "Evolutionist" in player_type else 35
                return {
                    player1_settings: gr.update(visible=True),
                    diff1: gr.update(visible=True, value="Medium"),
                    gen1: gr.update(visible=True, value=gens),
                    pop1: gr.update(visible=True, value=20),
                    algo1: "Genetic",
                    game_mode_selector: determine_game_mode(player_type, player2_type.value)
                }
            else:  # Random
                return {
                    player1_settings: gr.update(visible=True),
                    diff1: gr.update(visible=True, value="Easy"),
                    gen1: gr.update(visible=False),
                    pop1: gr.update(visible=False),
                    algo1: "Random",
                    game_mode_selector: determine_game_mode(player_type, player2_type.value)
                }

        def update_player2_settings(player_type):
            if player_type == "Human":
                return {
                    player2_settings: gr.update(visible=False),
                    diff2: gr.update(visible=False),
                    gen2: gr.update(visible=False),
                    pop2: gr.update(visible=False),
                    algo2: "Human",
                    game_mode_selector: determine_game_mode(player1_type.value, player_type)
                }
            elif "Minimax" in player_type:
                difficulty = "Easy" if "Novice" in player_type else "Medium" if "Strategist" in player_type else "Hard" if "Expert" in player_type else "Expert"
                return {
                    player2_settings: gr.update(visible=True),
                    diff2: gr.update(visible=True, value=difficulty),
                    gen2: gr.update(visible=False),
                    pop2: gr.update(visible=False),
                    algo2: "Minimax",
                    game_mode_selector: determine_game_mode(player1_type.value, player_type)
                }
            elif "Monte Carlo" in player_type:
                difficulty = "Easy" if "Explorer" in player_type else "Medium" if "Analyst" in player_type else "Hard" if "Strategist" in player_type else "Expert"
                return {
                    player2_settings: gr.update(visible=True),
                    diff2: gr.update(visible=True, value=difficulty),
                    gen2: gr.update(visible=False),
                    pop2: gr.update(visible=False),
                    algo2: "Monte Carlo",
                    game_mode_selector: determine_game_mode(player1_type.value, player_type)
                }
            elif "Genetic" in player_type:
                gens = 15 if "Pioneer" in player_type else 25 if "Evolutionist" in player_type else 35
                return {
                    player2_settings: gr.update(visible=True),
                    diff2: gr.update(visible=True, value="Medium"),
                    gen2: gr.update(visible=True, value=gens),
                    pop2: gr.update(visible=True, value=20),
                    algo2: "Genetic",
                    game_mode_selector: determine_game_mode(player1_type.value, player_type)
                }
            else:  # Random
                return {
                    player2_settings: gr.update(visible=True),
                    diff2: gr.update(visible=True, value="Easy"),
                    gen2: gr.update(visible=False),
                    pop2: gr.update(visible=False),
                    algo2: "Random",
                    game_mode_selector: determine_game_mode(player1_type.value, player_type)
                }

        def determine_game_mode(p1_type, p2_type):
            if p1_type == "Human" and p2_type == "Human":
                return "human_vs_human"
            elif p1_type == "Human" and p2_type != "Human":
                return "human_vs_ai"
            elif p1_type != "Human" and p2_type == "Human":
                return "ai_vs_human"
            else:
                return "ai_vs_ai"

        # Connect visibility updates
        player1_type.change(
            fn=update_player1_settings,
            inputs=[player1_type],
            outputs=[player1_settings, diff1, gen1, pop1, algo1, game_mode_selector]
        )

        player2_type.change(
            fn=update_player2_settings,
            inputs=[player2_type],
            outputs=[player2_settings, diff2, gen2, pop2, algo2, game_mode_selector]
        )

        # Update column button visibility when board size changes
        def update_column_visibility(cols):
            updates = []
            for i in range(10):
                updates.append(gr.update(visible=(i < cols)))
            return updates

        board_cols.change(
            fn=update_column_visibility,
            inputs=[board_cols],
            outputs=col_buttons
        )

        # Auto-play toggle
        auto_play_toggle.change(
            fn=toggle_auto_play,
            inputs=[auto_play_toggle],
            outputs=[]
        )

        # Event handlers with timer control
        def handle_game_start(rows, cols, mode, algo1, diff1, gen1, pop1, algo2, diff2, gen2, pop2):
            board, status, should_ai_play = setup_game(rows, cols, mode, algo1, diff1, gen1, pop1, algo2, diff2, gen2, pop2)
            return board, status, gr.Timer(value=0.8, active=should_ai_play)

        start_btn.click(
            fn=handle_game_start,
            inputs=[board_rows, board_cols, game_mode_selector,
                   algo1, diff1, gen1, pop1, algo2, diff2, gen2, pop2],
            outputs=[board_display, status_msg, timer]
        )

        # Column button events with timer control
        def handle_human_move(col):
            board, status, should_ai_play = make_human_move(col)
            return board, status, gr.Timer(value=0.8, active=should_ai_play and auto_play_active)

        for i, btn in enumerate(col_buttons):
            btn.click(
                fn=lambda col=i: handle_human_move(col),
                outputs=[board_display, status_msg, timer]
            )

        # AI move handlers
        def handle_ai_move():
            board, status, should_continue = make_ai_move()
            return board, status, gr.Timer(value=0.8, active=should_continue and auto_play_active)

        # Manual AI move button
        next_ai_btn.click(
            fn=handle_ai_move,
            outputs=[board_display, status_msg, timer]
        )

        # Timer tick handler
        timer.tick(
            fn=handle_ai_move,
            outputs=[board_display, status_msg, timer]
        )

        # Power-up button handlers
        def handle_power_up_opt1(col1, col2):
            board, status, should_ai_play = use_human_power_up(1, int(col1), int(col2))
            return board, status, gr.Timer(value=0.8, active=should_ai_play and auto_play_active)

        def handle_power_up_opt2(col):
            board, status, should_ai_play = use_human_power_up(2, int(col))
            return board, status, gr.Timer(value=0.8, active=should_ai_play and auto_play_active)

        def handle_beheading():
            board, status, should_ai_play = use_human_power_up('beheading', 0)
            return board, status, gr.Timer(value=0.8, active=should_ai_play and auto_play_active)

        power_btn_opt1.click(
            fn=handle_power_up_opt1,
            inputs=[power_col1_opt1, power_col2_opt1],
            outputs=[board_display, status_msg, timer]
        )

        power_btn_opt2.click(
            fn=handle_power_up_opt2,
            inputs=[power_col_opt2],
            outputs=[board_display, status_msg, timer]
        )

        beheading_btn.click(
            fn=handle_beheading,
            outputs=[board_display, status_msg, timer]
        )

        # Reset with timer stop
        def handle_reset():
            board, status = reset_game()
            return board, status, gr.Timer(active=False)

        reset_btn.click(
            fn=handle_reset,
            outputs=[board_display, status_msg, timer]
        )

        analysis_btn.click(fn=get_analysis, outputs=[analysis_display])

        # Auto-update analysis when board changes
        board_display.change(fn=get_analysis, outputs=[analysis_display])

    return demo

print("✅ User interface with automatic AI play and power-ups implemented successfully!")

# ======================== BLOCK 6: MAIN FUNCTION ========================

def main():
    """Main function to launch the Connect Four AI application"""
    print("\n" + "="*80)
    print("🎮 CONNECT FOUR AI - WITH POWER-UPS")
    print("="*80)
    print("🚀 Launching advanced Connect Four with multiple AI algorithms...")
    print("📊 Features: Minimax • Monte Carlo • Genetic Algorithm • Random Baseline")
    print("✨ NEW: Top Gone Power-Up - Remove opponent pieces strategically!")
    print("⚔️ NEW: Beheading Power-Up - Clear the entire board top!")
    print("🎯 Automatic AI play with 0.8s delay between moves")
    print("🔧 Toggle auto-play on/off during gameplay")
    print("="*80)

    # Create and launch the interface
    demo = create_interface()

    # Launch with optimized settings for Colab
    demo.launch(
        share=True,             # Create shareable link (important for Colab)
        debug=False,            # Disable debug mode for performance
        show_error=True,        # Show errors for debugging
        quiet=False             # Show startup information
        # Note: Removed server_name and server_port to let Gradio auto-select
    )

# Initialize game state
game = ConnectFourGUI()

print("✅ Complete Connect Four AI application with Top Gone and Beheading power-ups implemented!")
print("🎮 Automatic AI play enabled - AI moves after 0.8 seconds")
print("✨ Each player gets TWO power-ups per game")
print("🚀 Toggle auto-play on/off anytime during gameplay!")

# ======================== RUN THE APPLICATION ========================
if __name__ == "__main__":
    main()

🎮 Connect Four AI Tournament - Starting...
📦 Loading dependencies...
✅ Core game logic with FIXED win detection and Top Gone + Beheading power-ups implemented successfully!
✅ All AI algorithms with descriptive names and power-up support implemented successfully!
✅ Game management with automatic AI and power-up system implemented successfully!
✅ User interface with automatic AI play and power-ups implemented successfully!
✅ Complete Connect Four AI application with Top Gone and Beheading power-ups implemented!
🎮 Automatic AI play enabled - AI moves after 0.8 seconds
✨ Each player gets TWO power-ups per game
🚀 Toggle auto-play on/off anytime during gameplay!

🎮 CONNECT FOUR AI - WITH POWER-UPS
🚀 Launching advanced Connect Four with multiple AI algorithms...
📊 Features: Minimax • Monte Carlo • Genetic Algorithm • Random Baseline
✨ NEW: Top Gone Power-Up - Remove opponent pieces strategically!
⚔️ NEW: Beheading Power-Up - Clear the entire board top!
🎯 Automatic AI play with 0.8s delay betw